## Load Packages

In [1]:
suppressPackageStartupMessages({ 
    library(data.table) 
    library(dplyr) 
    library(ggplot2) 
    library(SingleCellExperiment)
    library(dplyr)
    library(celldex)
    library(SingleR)
    library(RColorBrewer)
    library(scater) 
    library(StabMap) 
    library(scran) 
    library(harmony) 
    library(patchwork)
    library(Seurat)
    library(plotly)
    library(pheatmap)
    library(batchelor)
    library(ggpubr)
    library(cowplot)
    library(viridis)
    library(patchwork)
    library(Cairo)
    library(grid)
    library(png)
    library(reshape2)  # Or use tidyr
    library(ggplot2)
    library(gridExtra)
    library(patchwork)
    library(grid)
    library(Cairo)
    library(png)
})

## Data Locations

In [2]:
##### Load in Seurat Objects for post QC analyses #####

io = list()
io$main = "/rds/project/rds-SDzz0CATGms/users/ltgh2" # this is the main directory

# Set the working directory
setwd(io$main)

# Load in the refined annotations
io$refined_annotations = file.path(io$main, "projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/outputs/FINAL_REFINED_ANNOTATIONS_REVISIONS.Rds") 

# Load in the pre_QC seqFISH OBJECT
io$seqFISH_preQC = file.path(io$main, "projects/03_seqFISH_FINAL/code/seqFISH_integration/processed/seqFISH_within_and_across_dataset_integration_without_neighbour_features_E6_7_8.Rds") 
io$seqFISH_preQC_meta = file.path(io$main, "projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/outputs/Figure_1_seurats/seqFISH_preQC_metadata.rds") 

# Load in the post_QC seqFISH OBJECT

io$seqFISH_postQC = file.path(io$main, "projects/03_seqFISH_FINAL/code/seqFISH_integration/processed/seqFISH_after_QC_filtering_within_and_across_dataset_integration_without_neighbour_features_E6_7_8.Rds") 
io$seqFISH_postQC_UMAP = file.path(io$main, "projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/outputs/Figure_1_seurats/postQC_seqFISH_umap_dimreduced.rds") 
io$seqFISH_postQC_pca = file.path(io$main, "projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/outputs/Figure_1_seurats/postQC_seqFISH_pca_dimreduced.rds")


## Load in the seqFISH Imputed Gene Expression
io$seqFISH_imputed_logcounts_e6 = file.path(io$main, 
                              "projects/03_seqFISH_FINAL/code/stabmap_imputation/output/seqFISH_e6_postQC_imputed_RNA.Rds")

io$seqFISH_imputed_logcounts_e7 = file.path(io$main, 
                              "projects/03_seqFISH_FINAL/code/stabmap_imputation/output/seqFISH_e7_postQC_imputed_RNA.Rds")

io$seqFISH_imputed_logcounts_e8_1 = file.path(io$main, 
                              "projects/03_seqFISH_FINAL/code/stabmap_imputation/output/seqFISH_e8_embryo1_postQC_imputed_RNA.Rds")

io$seqFISH_imputed_logcounts_e8_2 = file.path(io$main, 
                              "projects/03_seqFISH_FINAL/code/stabmap_imputation/output/seqFISH_e8_embryo2_postQC_imputed_RNA.Rds")

io$seqFISH_imputed_logcounts_e8_3 = file.path(io$main, 
                              "projects/03_seqFISH_FINAL/code/stabmap_imputation/output/seqFISH_e8_embryo3_postQC_imputed_RNA.Rds")

## Load in PreQC Seurat

In [3]:
# Load in the seqFISH object_postQC
seqFISH_preQC = readRDS(io$seqFISH_preQC)
seqFISH_preQC_meta = readRDS(io$seqFISH_preQC_meta)

seqFISH_preQC <- seqFISH_preQC[,seqFISH_preQC@meta.data$Estage %in% c("E6.5", "E7.5")]

# Add the MetaData to the Object
seqFISH_preQC <- AddMetaData(seqFISH_preQC, metadata = seqFISH_preQC_meta)

Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from shila_umap_ to shilaumap_”


## Load in PostQC Seurat

In [4]:
# Load in the seqFISH object_postQC
seqFISH_postQC = readRDS(io$seqFISH_postQC)
seqFISH_postQC_UMAP = readRDS(io$seqFISH_postQC_UMAP)
seqFISH_postQC_pca = readRDS(io$seqFISH_postQC_pca)

# Load in the imputed gene expression 
seqFISH_imputed_logcounts_e6 = readRDS(io$seqFISH_imputed_logcounts_e6)
seqFISH_imputed_logcounts_e7 = readRDS(io$seqFISH_imputed_logcounts_e7)
seqFISH_imputed_logcounts_e8_1 = readRDS(io$seqFISH_imputed_logcounts_e8_1)
seqFISH_imputed_logcounts_e8_2 = readRDS(io$seqFISH_imputed_logcounts_e8_2)
seqFISH_imputed_logcounts_e8_3 = readRDS(io$seqFISH_imputed_logcounts_e8_3)

# Create a combined matrix of gene expression across all time points
imputed_expression <- cbind(seqFISH_imputed_logcounts_e6$scRNA.RNA, 
                            seqFISH_imputed_logcounts_e7$scRNA.RNA, 
                            seqFISH_imputed_logcounts_e8_1$scRNA.RNA, 
                            seqFISH_imputed_logcounts_e8_2$scRNA.RNA, 
                            seqFISH_imputed_logcounts_e8_3$scRNA.RNA)

# Creat a new assay object to store imputed data
imp <- CreateAssayObject(data = imputed_expression)

# add this assay to the previously created Seurat object
seqFISH_postQC[["imputed"]] <- imp

# only keep the early embryos 
seqFISH_postQC <- seqFISH_postQC[,seqFISH_postQC@meta.data$Estage %in% c("E6.5", "E7.5")]

# add the PCs and the UMAP coordinates
seqFISH_postQC[["seqFISH_postQC_UMAP"]] <- seqFISH_postQC_UMAP
seqFISH_postQC[["seqFISH_postQC_pca"]] <- seqFISH_postQC_pca

# Add the MetaData to the Object
seqFISH_postQC <- AddMetaData(seqFISH_postQC, metadata = seqFISH_preQC_meta)

Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from shila_umap_ to shilaumap_”
Warning message:
“Cannot add objects with duplicate keys (offending key: UMAP_), setting key to 'seqfish_postqc_umap_'”
Warning message:
“Cannot add objects with duplicate keys (offending key: PC_), setting key to 'seqfish_postqc_pca_'”


## Load in the Refined Annotations

In [5]:
# Load in the refined_annotations
refined_annotations = readRDS(io$refined_annotations)

In [6]:
colnames(refined_annotations)

[1] "cell_ID"                                             
 [2] "orig.ident"                                          
 [3] "nCount_RNA"                                          
 [4] "nFeature_RNA"                                        
 [5] "seqFISH_celltype"                                    
 [6] "extended_atlas_celltype"                             
 [7] "AP"                                                  
 [8] "DV"                                                  
 [9] "dataset"                                             
[10] "cluster"                                             
[11] "subcluster"                                          
[12] "cluster_subcluster"                                  
[13] "stage"                                               
[14] "anatomy"                                             
[15] "UMAP_1"                                              
[16] "UMAP_2"                                              
[17] "celltype_PijuanSala2019"                             
[18] "majority_seqFISH_celltype"                           
[19] "majority_seqFISH_celltype_proportion"                
[20] "majority_extended_atlas_celltype"                    
[21] "majority_extended_atlas_celltype_proportion"         
[22] "majority_celltype_PijuanSala2019_celltype"           
[23] "majority_celltype_PijuanSala2019_celltype_proportion"
[24] "stage_transfer"                                      
[25] "poor_stage_alignment"                                
[26] "proportion_seqFISH_cells_per_subcluster"             
[27] "joint_clusters_pass_QC"                              
[28] "stage_alignment_pass_QC"                             
[29] "passed_QC"                                           
[30] "refined_annotation"                                  
[31] "embryo"

In [7]:
refined_annotations_ss <- subset(refined_annotations, select = c(refined_annotation, celltype_PijuanSala2019, extended_atlas_celltype))

In [8]:
seqFISH_postQC <- AddMetaData(seqFISH_postQC, metadata = refined_annotations_ss)
seqFISH_preQC <- AddMetaData(seqFISH_preQC, metadata = refined_annotations_ss)

## Load in R Scripts that Contain Cell Type Colours Etc.

In [9]:
script_files <- list.files("projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/R_scripts", 
                           pattern = "\\.[rR]$", full.names = TRUE)

invisible(lapply(script_files, source))

## Update Cell Type Labels

In [10]:
colnames(seqFISH_preQC_meta)

[1] "seqFISH_celltype"           "seqFISH_celltype_updated_2"
 [3] "dim3_binarized"             "AP"                        
 [5] "DV"                         "dim1"                      
 [7] "dim2"                       "dim3"                      
 [9] "STABMAP_UMAP_1"             "STABMAP_UMAP_2"            
[11] "embryo"

In [11]:
table(seqFISH_preQC_meta$embryo)


embryo_1 embryo_2 embryo_3 embryo_4 
    3811     4355     4545     2083 

In [12]:
str(seqFISH_preQC_meta$seqFISH_celltype_updated_2)

 Factor w/ 19 levels "LQ","low quality",..: 6 3 3 2 2 3 12 2 3 6 ...


In [13]:
df <- subset(seqFISH_preQC_meta, select = c(seqFISH_celltype_updated_2, embryo))

df <- df %>%
  mutate(seqFISH_celltype_updated_2 = as.character(seqFISH_celltype_updated_2)) %>%
  mutate(seqFISH_celltype_updated_2 = case_when(
    seqFISH_celltype_updated_2 == "LQ" & embryo == "embryo_2" ~ "Axial Mesendoderm",
    seqFISH_celltype_updated_2 == "LQ" & embryo == "embryo_1" ~ "Primitive Streak",
    seqFISH_celltype_updated_2 == "Haematoendothelial Progenitors #2" & embryo == "embryo_4" ~ "Caudal Epiblast",
    TRUE ~ seqFISH_celltype_updated_2  # Keep the remaining names unchanged
  ))

In [14]:
seqFISH_postQC <- AddMetaData(seqFISH_postQC, metadata = df)
seqFISH_preQC <- AddMetaData(seqFISH_preQC, metadata = df)

In [15]:
# Define the desired order of cell types
ordered_celltypes <- c(
  "low quality",
    
  "ExE Ectoderm Proximal #1",
  "ExE Ectoderm Proximal #2",
  
  "ExE Ectoderm Distal #1",
  "ExE Ectoderm Distal #2",

  "ExE Endoderm",
  "Visceral Endoderm #1",
  "Visceral Endoderm #2",
  "ExE Mesoderm",  
  "Mesodermal Wings",
  "Haematoendothelial Progenitors #1",
  "Haematoendothelial Progenitors #2",

  "Axial Mesendoderm",
  "Definitive Endoderm",
    

  "Rostral Ectoderm",
  "Primitive Streak",
  "Caudal Epiblast",
  "Epiblast"

)

# Convert to an ordered factor
seqFISH_preQC@meta.data$seqFISH_celltype_updated_2 <- factor(
  seqFISH_preQC@meta.data$seqFISH_celltype_updated_2,
  levels = ordered_celltypes,
  ordered = TRUE
)

# Check if ordering is correct
levels(seqFISH_preQC@meta.data$seqFISH_celltype_updated_2)

[1] "low quality"                       "ExE Ectoderm Proximal #1"         
 [3] "ExE Ectoderm Proximal #2"          "ExE Ectoderm Distal #1"           
 [5] "ExE Ectoderm Distal #2"            "ExE Endoderm"                     
 [7] "Visceral Endoderm #1"              "Visceral Endoderm #2"             
 [9] "ExE Mesoderm"                      "Mesodermal Wings"                 
[11] "Haematoendothelial Progenitors #1" "Haematoendothelial Progenitors #2"
[13] "Axial Mesendoderm"                 "Definitive Endoderm"              
[15] "Rostral Ectoderm"                  "Primitive Streak"                 
[17] "Caudal Epiblast"                   "Epiblast"

## Generate Main Figure 1 Figures

In [16]:
seqFISH_postQC$seqFISH_celltype_updated_2 <- factor(seqFISH_postQC$seqFISH_celltype_updated_2, 
                                                      levels = Harland_seqFISH_early_order)

In [17]:
seqFISH_postQC

An object of class Seurat 
28723 features across 9237 samples within 4 assays 
Active assay: across_timepoint_integration (351 features, 351 variable features)
 3 other assays present: originalexp, within_timepoint_integration, imputed
 7 dimensional reductions calculated: pca, umap, spatial, Shila_PCA, Shila_UMAP, seqFISH_postQC_UMAP, seqFISH_postQC_pca

## Create SFig1 H and I

In [18]:
seqFISH_preQC

An object of class Seurat 
1053 features across 14794 samples within 3 assays 
Active assay: across_timepoint_integration (351 features, 351 variable features)
 2 other assays present: originalexp, within_timepoint_integration
 5 dimensional reductions calculated: pca, umap, spatial, Shila_PCA, Shila_UMAP

In [19]:
seqFISH_postQC

An object of class Seurat 
28723 features across 9237 samples within 4 assays 
Active assay: across_timepoint_integration (351 features, 351 variable features)
 3 other assays present: originalexp, within_timepoint_integration, imputed
 7 dimensional reductions calculated: pca, umap, spatial, Shila_PCA, Shila_UMAP, seqFISH_postQC_UMAP, seqFISH_postQC_pca

## Spatial Plots with OriginalExp (seqFISH counts)

In [20]:
# Function to process originalexp data
process_original_exp <- function(seqFISH_object, exp_term = "originalexp") {
  exp_data <- as.data.frame(seqFISH_object@assays[[exp_term]]@data)
  exp_data <- t(exp_data) # Transpose the data
  exp_data <- as.data.frame(exp_data)
  exp_data$cell_ID <- rownames(exp_data)
  return(exp_data)
}

# Function to adjust dim1 and dim2 for all embryos
adjust_coordinates <- function(meta_data) {
  meta_data %>%
    group_by(embryo) %>%
    mutate(
      dim1 = dim1 - min(dim1, na.rm = TRUE), # Shift dim1
      dim2 = dim2 - min(dim2, na.rm = TRUE)  # Shift dim2
    ) %>%
    ungroup()
}

In [21]:
# Process seqFISH_postQC
seqFISH_postQC_originalexp <- process_original_exp(seqFISH_postQC, exp_term = "originalexp")
seqFISH_postQC_meta <- adjust_coordinates(seqFISH_postQC@meta.data)

# Process seqFISH_preQC
seqFISH_preQC_meta <- adjust_coordinates(seqFISH_preQC@meta.data)

# Merge meta data and processed originalexp data
merged_originalexp <- merge(seqFISH_postQC_meta, seqFISH_postQC_originalexp, by = "cell_ID")

In [22]:
plot_gene_expression <- function(merged_df, seqFISH_preQC_meta, embryo, gene, point_size = 1, text_size_2 = 10) {
  # Filter and sort data by gene expression level
  sorted_df <- merged_df[merged_df$embryo %in% c(embryo), ]
  sorted_df <- sorted_df[order(sorted_df[[gene]], decreasing = FALSE), ]  # Order by expression
  
  # Get the range of gene expression values
  gene_range <- range(sorted_df[[gene]], na.rm = TRUE)
    
  # Generate the plot
  p <- ggplot() +
    # Background points in grey for context
    geom_point(data = seqFISH_preQC_meta[seqFISH_preQC_meta$embryo %in% c(embryo), ], 
               aes(x = dim1, y = dim2), color = 'grey90', size = 3) +
    # Points colored by gene expression
    geom_point(data = sorted_df, aes(x = dim1, y = dim2, color = .data[[gene]]), size = point_size) +
    # Viridis color scale with min/max labels
    viridis::scale_color_viridis(breaks = gene_range, labels = round(gene_range, 0)) +
    coord_fixed() +
    theme_void() +
    facet_wrap(~embryo) +
    theme(
      legend.position = "right",
      strip.text = element_blank(),               # <- Removed facet titles safely
      legend.text = element_text(size = text_size_2),
      legend.title = element_text(size = text_size_2)
    ) +
    labs(color = NULL)

  return(p)
}



In [23]:
# Create a directory to save the plots
output_dir <- "projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig1h"
dir.create(output_dir, recursive = TRUE, showWarnings = FALSE)

# Your list of genes
genes_to_plot_sfig1h <- c("Slc7a3", "Lhx1", "Tbx3",
                          "Tal1", "Zic3", "Otx2",
                          "Soat2", "Col4a1", "Morc4",
                          "Cer1", "Fgfr2", "Tfap2c")

# Loop through genes and save each plot
for (gene in genes_to_plot_sfig1h) {
  plot <- plot_gene_expression(
    merged_df = merged_originalexp,
    seqFISH_preQC_meta = seqFISH_preQC_meta,
    embryo = c("embryo_1", "embryo_2"),
    gene = gene,
    point_size = 2,
    text_size_2 = 20
  )
  
  # Save plot
  output_path <- file.path(output_dir, paste0("SFig1h_", gene, ".png"))
  ggsave(filename = output_path, plot = plot, width = 5, height = 5, dpi = 300)
  message("Saved: ", output_path)
}

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig1h/SFig1h_Slc7a3.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig1h/SFig1h_Lhx1.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig1h/SFig1h_Tbx3.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig1h/SFig1h_Tal1.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig1h/SFig1h_Zic3.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig1h/SFig1h_Otx2.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_f

In [24]:
# Create a directory to save the plots
output_dir <- "projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig1i"
dir.create(output_dir, recursive = TRUE, showWarnings = FALSE)

# Your list of genes
genes_to_plot_sfig1i <- c("Tbx3", "Dlk1",
                         "Nkx1-2", "Cdx1",
                         "Lmo2", "Fgf3",
                         "Cxcl12", "Dnmt3l")

# Loop through genes and save each plot
for (gene in genes_to_plot_sfig1i) {
  plot <- plot_gene_expression(
    merged_df = merged_originalexp,
    seqFISH_preQC_meta = seqFISH_preQC_meta,
    embryo = c("embryo_3", "embryo_4"),
    gene = gene,
    point_size = 2,
    text_size_2 = 20
  )
  
  # Save plot
  output_path <- file.path(output_dir, paste0("SFig1h_", gene, ".png"))
  ggsave(filename = output_path, plot = plot, width = 5, height = 5, dpi = 300)
  message("Saved: ", output_path)
}

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig1i/SFig1h_Tbx3.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig1i/SFig1h_Dlk1.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig1i/SFig1h_Nkx1-2.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig1i/SFig1h_Cdx1.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig1i/SFig1h_Lmo2.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig1i/SFig1h_Fgf3.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_f

## Create SFig4D-G (Imputed Expression)

In [25]:
# Process seqFISH_postQC
seqFISH_postQC_imputed <- process_original_exp(seqFISH_postQC, exp_term = "imputed")
seqFISH_postQC_meta <- adjust_coordinates(seqFISH_postQC@meta.data)

# Process seqFISH_preQC
seqFISH_preQC_meta <- adjust_coordinates(seqFISH_preQC@meta.data)

# Merge meta data and processed originalexp data
merged_imputed <- merge(seqFISH_postQC_meta, seqFISH_postQC_imputed, by = "cell_ID")

In [26]:
# Create a directory to save the plots
output_dir <- "projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4d"
dir.create(output_dir, recursive = TRUE, showWarnings = FALSE)

# Your list of genes
genes_to_plot <- c("Ascl2", "Mesp1", "Tal1", "Sox17")

# Loop through genes and save each plot
for (gene in genes_to_plot) {
  plot <- plot_gene_expression(
    merged_df = merged_imputed,
    seqFISH_preQC_meta = seqFISH_preQC_meta,
    embryo = c("embryo_1", "embryo_2"),
    gene = gene,
    point_size = 2,
    text_size_2 = 20
  )
  
  # Save plot
  output_path <- file.path(output_dir, paste0("SFig1h_", gene, ".png"))
  ggsave(filename = output_path, plot = plot, width = 5, height = 5, dpi = 300)
  message("Saved: ", output_path)
}

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4d/SFig1h_Ascl2.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4d/SFig1h_Mesp1.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4d/SFig1h_Tal1.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4d/SFig1h_Sox17.png



In [27]:
# Create a directory to save the plots
output_dir <- "projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4e"
dir.create(output_dir, recursive = TRUE, showWarnings = FALSE)

# Your list of genes
genes_to_plot <- c("Gata1", "Perp", "Cxcl12", "Sox17")

# Loop through genes and save each plot
for (gene in genes_to_plot) {
  plot <- plot_gene_expression(
    merged_df = merged_imputed,
    seqFISH_preQC_meta = seqFISH_preQC_meta,
    embryo = c("embryo_3"),
    gene = gene,
    point_size = 2,
    text_size_2 = 20
  )
  
  # Save plot
  output_path <- file.path(output_dir, paste0("SFig1h_", gene, ".png"))
  ggsave(filename = output_path, plot = plot, width = 5, height = 5, dpi = 300)
  message("Saved: ", output_path)
}

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4e/SFig1h_Gata1.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4e/SFig1h_Perp.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4e/SFig1h_Cxcl12.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4e/SFig1h_Sox17.png



In [28]:
# Create a directory to save the plots
output_dir <- "projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4f"
dir.create(output_dir, recursive = TRUE, showWarnings = FALSE)

# Your list of genes
genes_to_plot <- c("Afp", "Foxa2", "Cer1", "Lefty2",
                  "Aplnr", "Hoxb1", "Fgf8", "Otx2")

# Loop through genes and save each plot
for (gene in genes_to_plot) {
  plot <- plot_gene_expression(
    merged_df = merged_imputed,
    seqFISH_preQC_meta = seqFISH_preQC_meta,
    embryo = c("embryo_1", "embryo_2"),
    gene = gene,
    point_size = 2,
    text_size_2 = 20
  )
  
  # Save plot
  output_path <- file.path(output_dir, paste0("SFig1h_", gene, ".png"))
  ggsave(filename = output_path, plot = plot, width = 5, height = 5, dpi = 300)
  message("Saved: ", output_path)
}

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4f/SFig1h_Afp.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4f/SFig1h_Foxa2.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4f/SFig1h_Cer1.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4f/SFig1h_Lefty2.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4f/SFig1h_Aplnr.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4f/SFig1h_Hoxb1.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6

In [30]:
# Create a directory to save the plots
output_dir <- "projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4g"
dir.create(output_dir, recursive = TRUE, showWarnings = FALSE)

# Your list of genes
genes_to_plot <- c("Bmp4", "Cdx1", "Otx2", "Hand1", "Fst", "Ptgis")

# Loop through genes and save each plot
for (gene in genes_to_plot) {
  plot <- plot_gene_expression(
    merged_df = merged_imputed,
    seqFISH_preQC_meta = seqFISH_preQC_meta,
    embryo = c("embryo_3"),
    gene = gene,
    point_size = 2,
    text_size_2 = 20
  )
  
  # Save plot
  output_path <- file.path(output_dir, paste0("SFig1h_", gene, ".png"))
  ggsave(filename = output_path, plot = plot, width = 5, height = 5, dpi = 300)
  message("Saved: ", output_path)
}

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4g/SFig1h_Bmp4.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4g/SFig1h_Cdx1.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4g/SFig1h_Otx2.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4g/SFig1h_Hand1.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4g/SFig1h_Fst.png

Saved: projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/GITHUB_UPLOAD_FOLDER/6_figure_generation/sfig_1_and_sfig_4/output/sfig4g/SFig1h_Ptgis.png



## Print out Package Versions

In [31]:
sessionInfo()

R version 4.2.1 (2022-06-23)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.5 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/liblapack.so.3

locale:
 [1] LC_CTYPE=C.UTF-8    LC_NUMERIC=C        LC_TIME=C          
 [4] LC_COLLATE=C        LC_MONETARY=C       LC_MESSAGES=C      
 [7] LC_PAPER=C          LC_NAME=C           LC_ADDRESS=C       
[10] LC_TELEPHONE=C      LC_MEASUREMENT=C    LC_IDENTIFICATION=C

attached base packages:
[1] grid      stats4    stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] gridExtra_2.3               reshape2_1.4.4             
 [3] png_0.1-8                   Cairo_1.6-0                
 [5] viridis_0.6.2               viridisLite_0.4.1          
 [7] cowplot_1.1.1               ggpubr_0.4.0               
 [9] batchelor_1.12.3            pheatmap_1.0.12            
[11] plotly_4.10.0 